In [ ]:
!pip install optuna

In [ ]:
!pip install lightgbm

In [ ]:
import shap
import optuna
import pandas as pd
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import numpy as np
from time import time
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

In [2]:
df_train = pd.read_parquet("/kaggle/input/data-tour/train.parquet")
df_train

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,flag
0,1678548,3,8,7,17,16,9,1,9,2,...,0,0,1,1,3,2,1,0,0,0
1,2834188,11,3,2,0,7,14,8,2,5,...,0,0,1,1,2,4,1,0,0,0
2,811902,11,9,6,11,13,14,8,2,5,...,3,3,4,1,2,4,1,0,0,0
3,836450,1,16,0,13,0,4,9,5,2,...,3,3,4,1,3,4,1,0,0,0
4,1769024,15,9,9,4,8,1,11,1,2,...,3,3,4,1,2,4,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17659828,2103608,4,1,9,4,8,1,11,10,2,...,3,3,4,1,2,3,1,1,1,0
17659829,2169551,7,15,4,12,10,6,13,15,2,...,0,0,1,1,3,2,1,0,0,0
17659830,1613436,1,8,9,16,5,9,9,10,3,...,3,3,4,1,3,4,1,0,0,0
17659831,1604231,6,17,13,12,1,2,4,2,2,...,0,0,1,1,3,0,1,0,0,0


In [3]:
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(["flag"],axis = 1), df_train.flag, stratify=df_train.flag, test_size=0.05, random_state = 42)
#del df_train
#_= gc.collect()

In [ ]:
# ⚙️ On réduit la taille des ensembles à 50 %
X_train_sub, _, y_train_sub, _ = train_test_split(
    X_train, y_train, train_size=0.4, stratify=y_train, random_state=42
)

X_val_sub, _, y_val_sub, _ = train_test_split(
    X_val, y_val, train_size=0.4, stratify=y_val, random_state=42
)

In [ ]:
def objective(trial):
    # --- Paramètres à optimiser ---
    params = {
        'booster': 'gbtree',
        'n_estimators': 3500,
        'learning_rate': 0.05,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'random_state': 42,
        'scale_pos_weight': (y_train.value_counts()[0] / y_train.value_counts()[1]),
        'eval_metric': 'auc',
        'use_label_encoder': False,

        # --- Hyperparamètres optimisés ---
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50)
    }

    # --- Entraînement ---
    model = XGBClassifier(**params)
    model.fit(
        X_train_sub, y_train_sub,
        eval_set=[(X_val_sub, y_val_sub)],
        early_stopping_rounds=100,
        verbose=False
    )

    # --- Prédiction et score ---
    y_pred = model.predict_proba(X_val_sub)[:, 1]
    auc = roc_auc_score(y_val_sub, y_pred)

    return auc

In [ ]:

# --- Lancement de l’optimisation bayésienne ---
start = time()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)
end = time()

print(f"\n✅ Optimisation terminée en {end - start:.1f} secondes")
print("Best AUC:", study.best_value)
print("Best parameters:", study.best_params)


In [ ]:
{'max_depth': 5, 'subsample': 0.995011247757305, 
 'colsample_bytree': 0.623863476660147, 'reg_alpha': 9.80184976286027, 
 'reg_lambda': 0.15121667196893812, 'min_child_weight': 38}

In [26]:
test_df_ready = pd.read_parquet("/kaggle/input/data-tour-new/test_df_ready.parquet")

In [ ]:
test_df_ready.columns

In [ ]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and col_type.name != 'category':
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                # entiers positifs
                if c_min >= 0:
                    if c_max < np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif c_max < np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif c_max < np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
            else:
                # flottants
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f"💾 Mémoire réduite de {start_mem:.2f} Mo à {end_mem:.2f} Mo ({100 * (start_mem - end_mem) / start_mem:.1f}% de gain)")
    return df

In [ ]:
test_df_ready = reduce_mem_usage(test_df_ready)

In [14]:
# ⚙️ On réduit la taille des ensembles à 50 %
X_train_sub, _, y_train_sub, _ = train_test_split(
    X_train, y_train, train_size=0.2, stratify=y_train, random_state=42
)

X_val_sub, _, y_val_sub, _ = train_test_split(
    X_val, y_val, train_size=0.2, stratify=y_val, random_state=42
)

In [15]:
start = time()

# ✅ Modèle de base avec tes paramètres fixes
model_xgb = XGBClassifier(
    booster='gbtree',
    n_estimators=1500,
    learning_rate=0.05,
    #max_depth=6,
    subsample=0.99,
    colsample_bytree=0.624,
    min_child_weight=30,
    tree_method='gpu_hist',       # GPU-accéléré
    predictor='gpu_predictor',
    random_state=42,
    scale_pos_weight=(y_val_sub.value_counts(normalize=True)[0] / y_val_sub.value_counts(normalize=True)[1]),
    eval_metric='auc'
)

# ✅ Grille de recherche — tu peux ajuster la granularité
param_grid = {
    'reg_alpha': [10, 20, 30],
    'reg_lambda': [50, 80, 120],
    "max_depth": [9, 10, 11],
    "min_child_weight" : [20, 30, 40]
}

# ✅ Configuration de la recherche
grid_search = GridSearchCV(
    estimator=model_xgb,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=2,
    n_jobs=-1  # utilise tous les cœurs disponibles
)

# ✅ Entraînement
grid_search.fit(
    X_train_sub, y_train_sub,
    eval_set=[(X_val_sub, y_val_sub)],
    early_stopping_rounds=150,
    verbose=100  # ✅ active les logs internes d'XGBoost
)


# ✅ Résultats
print("\nMeilleurs paramètres :", grid_search.best_params_)
print(f"Meilleur score AUC : {grid_search.best_score_:.4f}")

end = time()
print(f"Durée totale du GridSearch : {end - start:.2f} secondes")

Fitting 3 folds for each of 81 candidates, totalling 243 fits


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`se

[0]	validation_0-auc:0.60755
[0]	validation_0-auc:0.60850


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [20:55:08] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [20:55:08] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [20:55:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [20:55:08] WARNING: /workspace/src/learner.cc:742:

[0]	validation_0-auc:0.60847
[0]	validation_0-auc:0.60883
[100]	validation_0-auc:0.65011
[100]	validation_0-auc:0.64913
[100]	validation_0-auc:0.65033
[100]	validation_0-auc:0.65098
[200]	validation_0-auc:0.65354
[200]	validation_0-auc:0.65313
[200]	validation_0-auc:0.65427
[200]	validation_0-auc:0.65455
[300]	validation_0-auc:0.65444
[300]	validation_0-auc:0.65304
[300]	validation_0-auc:0.65533
[300]	validation_0-auc:0.65509
[385]	validation_0-auc:0.65178


...

[CV] END max_depth=11, min_child_weight=40, reg_alpha=30, reg_lambda=120; total time= 1.5min
[CV] END max_depth=11, min_child_weight=40, reg_alpha=30, reg_lambda=120; total time= 1.6min
[370]	validation_0-auc:0.65150


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [22:32:07] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[CV] END max_depth=11, min_child_weight=40, reg_alpha=30, reg_lambda=120; total time= 1.5min


`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
[22:32:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

[22:32:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.



[0]	validation_0-auc:0.60841
[100]	validation_0-auc:0.65122
[200]	validation_0-auc:0.65596
[300]	validation_0-auc:0.65702
[400]	validation_0-auc:0.65681
[475]	validation_0-auc:0.65657

Meilleurs paramètres : {'max_depth': 9, 'min_child_weight': 40, 'reg_alpha': 20, 'reg_lambda': 120}
Meilleur score AUC : 0.6548
Durée totale du GridSearch : 5887.71 secondes


In [21]:

start = time()

model_xgb4 = XGBClassifier(
    booster='gbtree',
    n_estimators=3500,
    learning_rate=0.05,
    max_depth=11,
    subsample=0.99,
    colsample_bytree=0.56,
    reg_alpha=20,
    reg_lambda=120,
    min_child_weight=40,
    tree_method='gpu_hist',      # ⚡ utilise le GPU
    predictor='gpu_predictor',   # ⚡ prédiction sur GPU aussi
    random_state=42,
    scale_pos_weight=(y_train.value_counts(normalize=True)[0] / y_train.value_counts(normalize=True)[1]),
    eval_metric='auc'
)

model_xgb4.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=150,
    verbose=100
)

end = time()
print(f"Training duration: {end - start:.2f} seconds")

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
[22:54:46] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

[22:54:46] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.



[0]	validation_0-auc:0.62787
[100]	validation_0-auc:0.66086
[200]	validation_0-auc:0.66538
[300]	validation_0-auc:0.66671
[400]	validation_0-auc:0.66734
[500]	validation_0-auc:0.66743
[600]	validation_0-auc:0.66725
[633]	validation_0-auc:0.66709
Training duration: 330.45 seconds


In [22]:
# prédiction sur le validation set
#val_preds_proba = model_xgb.predict_proba(X_val2, iteration_range=(0, model_xgb.best_iteration))[:, 1]

# prédiction sur le train set
train_preds_proba = model_xgb4.predict_proba(
    X_train,
    iteration_range=(0, model_xgb4.best_iteration)
)[:, 1].astype(np.float32)

train_auc = roc_auc_score(y_train, train_preds_proba)
#val_auc = roc_auc_score(y_val, val_preds_proba)
print(f"Train AUC: {train_auc}\n")

[23:01:11] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"



Train AUC: 0.723937861428088



In [34]:
import joblib

In [36]:
joblib.dump(model_xgb, "model_xgb_good.pickle")

['model_xgb_good.pickle']

In [24]:
val_preds_proba2 = model_xgb4.predict_proba(X_val)[:, 1]
val_auc2 = roc_auc_score(y_val, val_preds_proba2)
val_auc2

0.6674865614577457

In [ ]:
# Explainer basé sur le modèle et le dataset
explainer = shap.Explainer(model_xgb4, X_val)
shap_values = explainer(X_val)

In [ ]:
shap.summary_plot(shap_values[1], X_val)

In [ ]:
roc_auc_score(y_train, model_xgb4.predict_proba(X_train)[:, 1])

In [ ]:
# Importance des features
importance = model_xgb4.feature_importances_

# Si tu veux les associer aux noms de colonnes
feature_importance_df = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importance
}).sort_values(by="importance", ascending=False)

feature_importance_df.head(40)

In [ ]:
test_preds_proba = model_xgb4.predict_proba(test_df_ready[X_train.columns], iteration_range=(0, model_xgb.best_iteration))[:, 1]

In [42]:
test_preds_proba_df = pd.DataFrame({"target": test_preds_proba})
submission_df = pd.concat([test_df_ready ["id_x_rn"], test_preds_proba_df], axis = 1)
submission_df.rename(columns={"id_x_rn": "id"}, inplace=True)
submission_df

,id,target
0,1472943_x_2,0.525985
1,660465_x_5,0.451110
2,1788193_x_3,0.488672
3,2767146_x_3,0.368892
4,2601698_x_9,0.512741
...,...,...
654063,2159273_x_7,0.541461
654064,320081_x_7,0.332271
654065,2663677_x_4,0.537884
654066,1382804_x_18,0.441834


In [ ]:
submission_df[(submission_df["target"] <= 0.4) | (submission_df["target"] >= 0.6)]

In [ ]:
submission_df.to_parquet("sample_submission.parquet")